In [1]:
import numpy as np
import pandas as pd
import gc
import os
import matplotlib.pyplot as plt
import polars as pl
from sklearn.metrics import mean_squared_error, mean_absolute_error
from joblib import Parallel, delayed
from more_itertools import chunked
from functools import reduce
from typing import List
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
import pandas as pd
import joblib
import os
import torch.nn as nn

In [2]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")

False
0
No GPU


/opt/conda/lib/python3.10/site-packages/torch/cuda/__init__.py:628: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [3]:
df_full = pd.read_parquet('./data/product_train_val_NN_TORCH.parquet', engine='fastparquet')

In [4]:
# Separar conjuntos
df_train = df_full[df_full['PERIODO'] <= 201909].copy()
df_val = df_full[(df_full['PERIODO'] == 201910)].copy()
df_pred = df_full[df_full['PERIODO'] == 201912].copy()
del df_full
gc.collect()

20

In [5]:
import joblib

target_col = 'CLASE_LOG1P_Z'
cat_cols = ['ID_CAT1', 'ID_CAT2', 'ID_CAT3', 'ID_BRAND', 'SKU_SIZE', 'MES_PROBLEMATICO', 'PRODUCT_RANK_BIN', 
            'IS_FEBRERO', 'ESTOY_PREDICIENDO_FEBRERO', 'CAIDA_ABRUPTA']
label_encoders = {}

# Cargar los encoders entrenados
for col in cat_cols:
    le = joblib.load(f'encoders/{col}_encoder.pkl')
    label_encoders[col] = le

    # Transformar los datasets (train, val, pred) usando ese encoder
    for df in [df_train, df_val, df_pred]:
        df[col] = df[col].map(lambda x: le.transform([x])[0] if x in le.classes_ else 0)

# Definir embedding_sizes (lo mejor es usar el encoder para ver la cantidad de clases)
embedding_sizes = [
    (len(label_encoders[col].classes_) + 1, min(50, (len(label_encoders[col].classes_) + 1) // 2))
    for col in cat_cols
]

# Excluir columnas que no deben ir al modelo
excluir = ['PERIODO', 'CUSTOMER_ID', 'PRODUCT_ID', 'CLASE_LOG1P_Z', 'ORDINAL']
feature_cols = [col for col in df.columns if col not in excluir and col not in cat_cols]

In [7]:
WINDOW_SIZE = 12
PREDICT_HORIZON = 2

X_seq_train, y_seq_train, prod_ids_train = [], [], []

for prod, df_prod in df_train.groupby('PRODUCT_ID'):
    df_prod = df_prod.sort_values('PERIODO')
    for i in range(len(df_prod) - WINDOW_SIZE - PREDICT_HORIZON):
        seq = df_prod.iloc[i:i+WINDOW_SIZE][feature_cols].values
        target = df_prod.iloc[i+WINDOW_SIZE+PREDICT_HORIZON][target_col]
        X_seq_train.append(seq)
        y_seq_train.append(target)
        prod_ids_train.append(prod)

X_seq_train = np.array(X_seq_train)
y_seq_train = np.array(y_seq_train)
prod_ids_train = np.array(prod_ids_train)


In [8]:
X_seq_val, y_seq_val, prod_ids_val = [], [], []

for prod, df_prod in df_val.groupby('PRODUCT_ID'):
    df_prod = df_prod.sort_values('PERIODO')
    for i in range(len(df_prod) - WINDOW_SIZE - PREDICT_HORIZON):
        seq = df_prod.iloc[i:i+WINDOW_SIZE][feature_cols].values
        target = df_prod.iloc[i+WINDOW_SIZE+PREDICT_HORIZON][target_col]
        X_seq_val.append(seq)
        y_seq_val.append(target)
        prod_ids_val.append(prod)

X_seq_val = np.array(X_seq_val)
y_seq_val = np.array(y_seq_val)
prod_ids_val = np.array(prod_ids_val)


In [10]:
for prod, df_prod in df_pred.groupby('PRODUCT_ID'):
    df_hist = pd.concat([
        df_train[df_train['PRODUCT_ID'] == prod],
        df_val[df_val['PRODUCT_ID'] == prod],
        df_prod
    ]).sort_values('PERIODO')
    idxs = df_hist.index[df_hist['PERIODO'] == 201910].tolist()
    if not idxs:
        continue
    idx = idxs[0]
    start = idx - WINDOW_SIZE + 1
    if start < 0:
        continue
    seq = df_hist.iloc[start:idx+1][feature_cols].values
    # --- CHEQUEO ---
    if seq.shape != (WINDOW_SIZE, len(feature_cols)):
        print(f"Producto {prod} - shape inválido: {seq.shape}, saltando")
        continue
    X_seq_pred.append(seq)
    prod_ids_pred.append(prod)



Producto 20002 - shape inválido: (8, 211), saltando
Producto 20003 - shape inválido: (0, 211), saltando
Producto 20004 - shape inválido: (0, 211), saltando
Producto 20005 - shape inválido: (0, 211), saltando
Producto 20006 - shape inválido: (0, 211), saltando
Producto 20007 - shape inválido: (0, 211), saltando
Producto 20008 - shape inválido: (0, 211), saltando
Producto 20009 - shape inválido: (0, 211), saltando
Producto 20010 - shape inválido: (0, 211), saltando
Producto 20011 - shape inválido: (0, 211), saltando
Producto 20012 - shape inválido: (0, 211), saltando
Producto 20013 - shape inválido: (0, 211), saltando
Producto 20014 - shape inválido: (0, 211), saltando
Producto 20015 - shape inválido: (0, 211), saltando
Producto 20016 - shape inválido: (0, 211), saltando
Producto 20017 - shape inválido: (0, 211), saltando
Producto 20018 - shape inválido: (0, 211), saltando
Producto 20019 - shape inválido: (0, 211), saltando
Producto 20020 - shape inválido: (0, 211), saltando
Producto 200

In [11]:
print(f"Cantidad de productos listos para predecir: {len(X_seq_pred)}")


Cantidad de productos listos para predecir: 779


In [13]:
print(df_train.columns.tolist())

['PERIODO', 'MES_SIN', 'MES_COS', 'ID_CAT1', 'ID_CAT2', 'ID_CAT3', 'ID_BRAND', 'SKU_SIZE', 'PRODUCT_ID', 'MES_PROBLEMATICO', 'IS_FEBRERO', 'ESTOY_PREDICIENDO_FEBRERO', 'CAIDA_ABRUPTA', 'CLASE_LOG1P_Z', 'PRODUCT_RANK_BIN', 'ORDINAL_Z', 'ANIO_Z', 'MES_Z', 'TRIMESTRE_Z', 'PRODUCT_ID_Z', 'TN_Z', 'PROM_ULT_3_FEBREROS_Z', 'DIF_TN_VS_FEBREROS_ULT_3_Z', 'TN_MAX_HISTORICO_Z', 'TN_DIST_A_MAX_HIST_Z', 'TN_RATIO_VS_MAX_HIST_Z', 'TN_MEAN_03_Z', 'PENDIENTE_TENDENCIA_3_Z', 'TN_EWMA_03_Z', 'TN_MEDIAN_03_Z', 'TN_MIN_03_Z', 'TN_MAX_03_Z', 'TN_STD_03_Z', 'TN_SKEW_03_Z', 'TN_KURT_03_Z', 'TN_GROWTH_03_Z', 'TN_IQR_03_Z', 'TN_SUM_03_Z', 'TN_COUNT_POS_03_Z', 'TN_PCT_ZERO_03_Z', 'TN_LAST_03_Z', 'TN_LAST_DIFF_03_Z', 'TN_COEF_VAR_3_Z', 'TN_MAXMIN_RATIO_3_Z', 'TN_RANGO_3_Z', 'TN_RANGO_REL_3_Z', 'TN_LAST_VS_MEDIAN_3_Z', 'TN_CHANGE_PREV_WINDOW_3_Z', 'TN_ZEROS_END_3_Z', 'TN_LAST_PCT_SUM_3_Z', 'TN_PCT90_3_Z', 'TN_PCT10_3_Z', 'TN_PCT_WIDTH_3_Z', 'TN_MINUS_MEAN_03_Z', 'TN_MINUS_MEDIAN_03_Z', 'TN_MINUS_EWMA_03_Z', 'TN_O

In [16]:
import numpy as np
scaler_y = joblib.load('scaler_y_CLASE_LOG1P.joblib')
# 1. Media histórica de CLASE_LOG1P_Z por producto (en df_train)
medias_log1p_z = df_train.groupby('PRODUCT_ID')['CLASE_LOG1P_Z'].mean().to_dict()

# 2. Para los productos sin historia:
# Arma array y deshace el Z-score usando el scaler_y
valores_z = np.array([medias_log1p_z.get(prod, 0) for prod in sin_historia]).reshape(-1, 1)
# Deshacer z-score: vuelve a log1p(TN)
valores_log1p = scaler_y.inverse_transform(valores_z).flatten()
# Vuelve a TN real
valores_tn = np.expm1(valores_log1p)

# 3. Resultado final como dict {PRODUCT_ID: TN_predicho}
predicciones_simple = dict(zip(list(sin_historia), valores_tn))

display(predicciones_simple)


{20001: 1400.6702005677594}